[View in Colaboratory](https://colab.research.google.com/github/robmoore/diabetes-prediction/blob/master/Create_RxNorm_SQLite_Database.ipynb)

In [0]:
!apt install sqlite3 -q

In [0]:
import getpass
import sqlite3
import pandas as pd
import numpy as np

from contextlib import closing

# Data uploaded to Google Drive below. Assumes the existence of a folder named 'Diabetes-Prediction' in your Google Drive account.
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
![ -f "terminology_download_script.zip" ] || (wget -q -N https://download.nlm.nih.gov/rxnorm/terminology_download_script.zip && unzip -o terminology_download_script.zip curl-uts-download.sh uts.nlm.nih.gov.crt && chmod +x curl-uts-download.sh)

In order to download the RxNorm data required below, you'll need to have an [account](https://uts.nlm.nih.gov//license.html). Please enter your credentials below.

In [0]:
uts_username = getpass.getpass(prompt='Enter your UTS username: ')
uts_password = getpass.getpass(prompt='Enter your UTS password: ')
# Insert username and password and fix issue with Windows variable reference so logout occurs
!sed -i'' "s/UTS_USERNAME=.*$/UTS_USERNAME=\"{uts_username}\"/;s/UTS_PASSWORD=.*$/UTS_PASSWORD=\"{uts_password}\"/;s/%\([^%]*\)%/\$\1/g" curl-uts-download.sh

In [0]:
rxnorm_fn='RxNorm_full_current.zip'
![ -f {rxnorm_fn} ] || (./curl-uts-download.sh https://download.nlm.nih.gov/umls/kss/rxnorm/{rxnorm_fn} > /dev/null && zip -T {rxnorm_fn})

In [0]:
!rm rxnorm.db

In [0]:
%%bash
if [ ! -d "rrf" ]; then
  echo "Unzipping RxNorm file"
  unzip -o RxNorm_full_current.zip rrf/RXNCONSO.RRF rrf/RXNREL.RRF rrf/RXNSAT.RRF scripts/mysql/*.sql
fi

# Mostly stolen from https://github.com/chb/py-umls/blob/master/databases/rxnorm.sh
if [ ! -e "rxnorm.db" ]; then
  echo "Creating tables"
  sqlite3 rxnorm.db < scripts/mysql/Table_scripts_mysql_rxn.sql

  for f in rrf/*.RRF; do
    echo "-> Pre-processing ${f}"
    # strip last pipe to avoid warnings, escape quotes with double quotes and also by wrapping all fields in quotes
    sed 's/|$//;s/"/""/g;s/[^|]*/"&"/g' "$f" > "$f.prep"
    table=$(basename "${f%.RRF}")
    echo "-> Importing $table"
    sqlite3 rxnorm.db ".import '$f.prep' '$table'"
  done

  echo "Creating indicies"
  sqlite3 rxnorm.db < scripts/mysql/Indexes_mysql_rxn.sql

  echo "Dropping unused tables"
  grep DROP scripts/mysql/Table_scripts_mysql_rxn.sql | grep -v 'RXNREL\|RXNSAT\|RXNCONSO' | sqlite3 rxnorm.db
fi

In [0]:
def processNdcs(ndcs):
  atvs = (','.join('"{0}"'.format(ndc) for ndc in ndcs))
  with closing(sqlite3.connect('file:rxnorm.db?mode=ro', uri=True)) as conn: 
    return pd.read_sql_query("""
SELECT rel1.RXCUI2 AS RXCUI, rel2.RXCUI1 AS INGREDIENT
FROM RXNREL rel1
JOIN RXNCONSO con ON rel1.RXCUI1 = con.RXCUI AND TTY='SCDC'
JOIN RXNREL rel2 ON rel1.RXCUI1 = rel2.RXCUI2 AND rel2.RELA = 'has_ingredient'
WHERE rel1.RXCUI2 IN (SELECT RXCUI FROM RXNSAT WHERE ATN = 'NDC' AND ATV IN ({})) AND rel1.RELA = 'consists_of' 
ORDER BY RXCUI
""".format(atvs), conn)

In [0]:
ndcs = ["23490544301", "00143126730", "00456069801", "60432060504", "00247211730", "00247028100", "00093084015", "00168000215", "00173045301"]

ingredients = processNdcs(ndcs)
print(ingredients)

In [0]:
!rm -rf rrf scripts curl-uts-* terminology_* uts* {rxnorm_fn}
!cp rxnorm.db /content/gdrive/My\ Drive/Diabetes-Prediction/